Template for TAPT Training

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

In [ ]:
from datasets import load_dataset
dataset_name = "BigTMiami/amazon_helpfulness"
dataset = load_dataset(dataset_name)

train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
import numpy as np
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)
from sklearn.metrics import accuracy_score,  f1_score

from datasets import load_dataset

# Base Roberta

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM
base_model = AutoModelForMaskedLM.from_pretrained("roberta-base")
print_gpu_utilization()

In [ ]:
# Training params
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="helpfulness_tapt_base",
    learning_rate=0.0001, # Paper for TAPT training
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=16,
    num_train_epochs=100, # Just one epoch for one pass on data
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    #learning_rate_decay="linear",
    evaluation_strategy="steps",
    eval_steps=500, # Evaluates every 50 steps
    save_strategy="steps",
    save_steps=500, # Checkpoint saves every 500 steps / every 1,000,000 reviews ( 500 * 10 * 34 * 6)
    save_total_limit=2, # Saves latest 2 checkpoints
    torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="steps", # Is default
    logging_steps=50, # Logs training progress
)

In [ ]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["eval"],
    data_collator=data_collator,
)

In [ ]:
eval_results = trainer.evaluate()
non_trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Non Trained Eval Loss: {non_trained_eval_loss:.2f}")
print(eval_results)

In [ ]:
results = trainer.train()
print_gpu_utilization()
print(results)

In [ ]:
eval_results = trainer.evaluate()
trained_eval_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Trained Eval Loss: {trained_eval_loss:.2f}")
print(eval_results)

In [ ]:
trainer.push_to_hub()

# Seq Bn Architecture

# Pfeiffer Architecture

# Houlsby Architecture